## Testing different functions and classes

In [1]:
from watermodelCLASS import WaterModel
import xarray as xr
import numpy as np
import time

Testing on the Bentfeld Catchment

## Testing Watermodel

In [2]:
data_path = r"data/catchments/GRDC-Daily.nc"

runoff = xr.open_mfdataset(data_path).load()
runoff = runoff.sel(time=slice('2000-03-01', '2022-12-19'))

bentfeld_id = runoff["station_name"].values == "BENTFELD"
bentfeld_index = np.where(bentfeld_id)[0][0]

# Select only that timeseries
runoffBentfeld = runoff["runoff_mean"].isel(id=bentfeld_index)

bentfeld_path = r"data/catchment_timeseries/bentfeld"

temperature = xr.open_mfdataset(bentfeld_path + '/temperature.nc', combine='by_coords').load()
precipitation = xr.open_mfdataset(bentfeld_path + '/precipitation.nc', combine='by_coords').load()
radiation = xr.open_mfdataset(bentfeld_path + '/radiation.nc', combine='by_coords').load()
ndvi = xr.open_mfdataset(bentfeld_path + '/ndvi.nc', combine='by_coords').load()

data = xr.Dataset()
data['temperature'] = temperature['t2m']
data['precipitation'] = precipitation['precipitation']
data['radiation'] = radiation['nr']
data['ndvi'] = ndvi['ndvi']
data['observedRunoff'] = runoffBentfeld.broadcast_like(radiation['nr'])

#get rid of anythin before 2000 and after 2022
data = data.sel(time=slice('2000-03-01', '2022-12-19'))
data

<xarray.Dataset> Size: 7MB
Dimensions:         (time: 8329, x: 5, y: 5)
Coordinates:
  * time            (time) datetime64[ns] 67kB 2000-03-01 ... 2022-12-19
  * x               (x) float64 40B 8.55 8.65 8.75 8.85 8.95
  * y               (y) float64 40B 51.85 51.75 51.65 51.55 51.45
    spatial_ref     int64 8B 0
    id              int32 4B 6335082
Data variables:
    temperature     (time, y, x) float32 833kB nan nan nan ... 276.1 nan nan
    precipitation   (time, y, x) float64 2MB nan nan nan ... 0.004961 nan nan
    radiation       (time, y, x) float64 2MB nan nan nan ... -2.438e+04 nan nan
    ndvi            (time, y, x) float64 2MB nan nan nan ... 0.6008 nan nan
    observedRunoff  (time, y, x) float32 833kB 33.08 33.08 33.08 ... nan nan nan

In [3]:
dataMean = xr.Dataset()
dataMean['temperature'] = data["temperature"].mean(dim=("x", "y"))
dataMean['precipitation'] = data["precipitation"].mean(dim=("x", "y"))
dataMean['radiation'] = data["radiation"].mean(dim=("x", "y"))
dataMean['ndvi'] = data["ndvi"].mean(dim=("x", "y"))
dataMean['observedRunoff'] = data["observedRunoff"].sum(dim=("x", "y"))  
params_new = {
    "c_s": 840,
    "alpha": 8,
    "gamma": 0.5,
    "beta": 0.8,
    "c_m": 2.0,
    "temp_w": 0.75,
    "ndvi_w": 0.5,
    "iota": 10,
    "theta": 5
}

With the njit library the first run takes like 10 seconds to render, afterwards it can run the model like instant

In [6]:
start_time = time.time()
wm = WaterModel(params=params_new, data= dataMean)
runoff = wm.run_simulation_whole_catchment()
duration = time.time() - start_time
print(f"calculations completed in {duration:.5f} seconds.")

calculations completed in 0.00000 seconds.


## Testing Calibration